In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import ElasticNet
from sklearn.decomposition import PCA
# from catboost import CatBoostRegressor
from scipy.sparse import random
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from scipy.sparse import random

In [ ]:
df = pd.read_csv('kitchenham.csv')
df.head(5)

,Project,Client.code,Project.type,Actual.start.date,Actual.duration,Actual.effort,Adjusted.function.points,Estimated.completion.date,First.estimate,First.estimate.method
0,1,1,A,10-12-1996,107,485,101.65,15-04-1997,495,EO
1,2,1,D,16-06-1997,144,990,57.12,19-08-1997,1365,A
2,3,1,D,01-03-1997,604,13635,1010.88,30-06-1998,8244,EO
3,4,1,P,23-06-1997,226,1576,45.60,06-03-1998,1595,D
4,5,1,D,20-01-1997,326,3826,1022.58,01-01-1998,3828,A


#TRAIN TEST DATASET

In [ ]:

X=df[['Actual.duration', 'Adjusted.function.points', 'First.estimate']]
y=df[['Actual.effort']]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)


# FEATURE SCALING

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Project                    145 non-null    int64  
 1   Client.code                145 non-null    int64  
 2   Project.type               145 non-null    object 
 3   Actual.start.date          145 non-null    object 
 4   Actual.duration            145 non-null    int64  
 5   Actual.effort              145 non-null    int64  
 6   Adjusted.function.points   145 non-null    float64
 7   Estimated.completion.date  145 non-null    object 
 8   First.estimate             145 non-null    int64  
 9   First.estimate.method      145 non-null    object 
dtypes: float64(1), int64(5), object(4)
memory usage: 11.5+ KB


In [ ]:
df.describe()

,Project,Client.code,Actual.duration,Actual.effort,Adjusted.function.points,First.estimate
count,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000
mean,73.000000,2.103448,206.448276,3113.117241,527.669310,2855.972414
std,42.001984,0.863811,134.092005,9598.008922,1521.994936,6789.287453
min,1.000000,1.000000,37.000000,219.000000,15.360000,121.000000
25%,37.000000,2.000000,123.000000,874.000000,126.420000,898.000000
50%,73.000000,2.000000,171.000000,1557.000000,260.950000,1750.000000
75%,109.000000,2.000000,250.000000,2895.000000,603.580000,2879.000000
max,145.000000,6.000000,946.000000,113930.000000,18137.480000,79870.000000


#Stacked PCR

In [ ]:
from sklearn.decomposition import PCA


# Apply PCA for dimensionality reduction
n_components = 3  # Adjust the number of components as needed
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Define base regressors
base_regressors = [
    ('linear_regression', LinearRegression()),
    ('svr', SVR(kernel='linear')),
    ('random_forest', RandomForestRegressor(n_estimators=50, random_state=0))
]

# Define the meta-regressor
meta_regressor = LinearRegression()

# Create the stacked regressor
stacked_regressor = StackingRegressor(estimators=base_regressors, final_estimator=meta_regressor)

# Fit the model to the training data
stacked_regressor.fit(X_train_pca, y_train)

# Make predictions on the test set
y_pred = stacked_regressor.predict(X_test_pca)


def mean_absoiute_error(y_true, y_pred):

    mae = mean_absolute_error(y_true, y_pred)
    maximum_value = 100000
    minimum_value = 0
    converted_value = mae / (maximum_value - minimum_value)
    return converted_value

def r_mse(y_true, y_pred):

    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    maximum_value = 100000
    minimum_value = 0
    converted_value = rmse / (maximum_value - minimum_value)
    return converted_value

def r_squared(y_true, y_pred):

    r2 = r2_score(y_true, y_pred)
    maximum_value = 1
    minimum_value = 0
    converted_value = r2 / (maximum_value - minimum_value)
    return converted_value
# Calculate evaluation metrics
mae = mean_absoiute_error(y_test, y_pred)
rmse = r_mse(y_test, y_pred) # RMSE
r2 = r_squared(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Absolute Error (MAE): 0.006387321087779003
Root Mean Squared Error (RMSE): 0.008841063626704466
R-squared (R2) Score: 0.6970095718230628


#Linear Regression
and hyper parameter tunuing

In [ ]:

# Hyperparameter Tuning
param_grid = {
    'fit_intercept': [True, False]

}

grid_search = GridSearchCV(LinearRegression(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

# Best parameters found
print("Best Parameters:", grid_search.best_params_)

# Evaluating the model with best parameters
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)

# Evaluation Metrics
mae = mean_absoiute_error(y_test, y_pred)
rmse = r_mse(y_test, y_pred) # RMSE
r2 = r_squared(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

Best Parameters: {'fit_intercept': True}
Mean Absolute Error (MAE): 0.005641078045068945
Root Mean Squared Error (RMSE): 0.007810851524627361
R-squared (R2) Score: 0.7635078979146811


#Support Vector Machine

In [ ]:
from sklearn.svm import SVR
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

support=SVR(kernel='rbf')
support.fit(X_train,y_train)
y_pred=support.predict(X_test)


# Evaluation Metrics
mae = mean_absoiute_error(y_test, y_pred)
rmse = r_mse(y_test, y_pred) # RMSE
r2 = r_squared(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


Mean Absolute Error (MAE): 0.1995941793420417
Root Mean Squared Error (RMSE): 0.3328609657434054
R-squared (R2) Score: -0.14377649106868207


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#Random Forest Regression

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)
for i in np.arange(100,1100,100):
    random = RandomForestRegressor(n_estimators=i,max_features=5)
    random.fit(X_train, y_train)
    y_pred = random.predict(X_test)

# Evaluation Metrics
mae = mean_absoiute_error(y_test, y_pred)
rmse = r_mse(y_test, y_pred) # RMSE
r2 = r_squared(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

<ipython-input-6-bcf9f0a7e733>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random.fit(X_train, y_train)
<ipython-input-6-bcf9f0a7e733>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random.fit(X_train, y_train)
<ipython-input-6-bcf9f0a7e733>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random.fit(X_train, y_train)
<ipython-input-6-bcf9f0a7e733>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random.fit(X_train, y_train)
<ipython-input-6-bcf9f0a7e733>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Plea

Mean Absolute Error (MAE): 0.010733592068965516
Root Mean Squared Error (RMSE): 0.019013679567842095
R-squared (R2) Score: 0.7144696450675052


#KNeighbor


In [ ]:

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=50)

# Instantiate and fit the KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=3, weights='uniform')
neigh.fit(X_train, y_train)

# Predict on the test data
y_pred = neigh.predict(X_test)

# Calculate evaluation metrics
mae = mean_absoiute_error(y_test, y_pred)
rmse = r_mse(y_test, y_pred) # RMSE
r2 = r_squared(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


Mean Absolute Error (MAE): 0.05498125
Root Mean Squared Error (RMSE): 0.09310509183175752
R-squared (R2) Score: 0.8520215365341675


#Bagging

DecisionTreeRegressor(random_state=22)

In [ ]:
from sklearn.ensemble import BaggingRegressor

# Replace DecisionTreeRegressor with BaggingRegressor
bagging_dtree = BaggingRegressor(base_estimator=DecisionTreeRegressor(random_state=22),
                                 n_estimators=10,  # You can adjust the number of base estimators (trees)
                                 random_state=22)
bagging_dtree.fit(X_train, y_train)
y_pred = bagging_dtree.predict(X_test)

# Calculate evaluation metrics
mae = mean_absoiute_error(y_test, y_pred)
rmse = r_mse(y_test, y_pred) # RMSE
r2 = r_squared(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

Mean Absolute Error (MAE): 0.3493756756756757
Root Mean Squared Error (RMSE): 1.6233453505946593
R-squared (R2) Score: 0.20741309813519193


#Boosting

In [ ]:

gradient_booster = GradientBoostingRegressor(learning_rate=0.1)
gradient_booster.fit(X_train,y_train)
y_pred = gradient_booster.predict(X_test)


# Calculate evaluation metrics
mae = mean_absoiute_error(y_test, y_pred)
rmse = r_mse(y_test, y_pred) # RMSE
r2 = r_squared(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


Mean Absolute Error (MAE): 0.3503250939801526
Root Mean Squared Error (RMSE): 1.6665678306327936
R-squared (R2) Score: 0.1646450758464878


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#CATBOOST

In [ ]:

from catboost import CatBoostRegressor

# Instantiate the CatBoostRegressor model
catboost_model = CatBoostRegressor(iterations=100, depth=8, learning_rate=0.1, loss_function='MAE', random_seed=0)

# Fit the model to the training data
catboost_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = catboost_model.predict(X_test_scaled)

# Calculate evaluation metrics
mae = mean_absoiute_error(y_test, y_pred)
rmse = r_mse(y_test, y_pred) # RMSE
r2 = r_squared(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


0:	learn: 1277.5759236	total: 48.2ms	remaining: 4.78s
1:	learn: 1183.2274022	total: 50.1ms	remaining: 2.46s
2:	learn: 1122.8436978	total: 50.9ms	remaining: 1.65s
3:	learn: 1064.7459788	total: 52.4ms	remaining: 1.26s
4:	learn: 1005.8329741	total: 53.9ms	remaining: 1.02s
5:	learn: 925.2589670	total: 55.4ms	remaining: 868ms
6:	learn: 874.0298955	total: 56.8ms	remaining: 755ms
7:	learn: 829.7934343	total: 58.2ms	remaining: 669ms
8:	learn: 784.7045146	total: 59.6ms	remaining: 603ms
9:	learn: 730.9441425	total: 61ms	remaining: 549ms
10:	learn: 688.3737233	total: 62.5ms	remaining: 506ms
11:	learn: 654.2599557	total: 63.9ms	remaining: 468ms
12:	learn: 613.5181553	total: 65.3ms	remaining: 437ms
13:	learn: 589.9470358	total: 66.8ms	remaining: 410ms
14:	learn: 572.3547063	total: 68.1ms	remaining: 386ms
15:	learn: 553.3368961	total: 69.5ms	remaining: 365ms
16:	learn: 537.8402832	total: 71.2ms	remaining: 348ms
17:	learn: 519.9831065	total: 72.7ms	remaining: 331ms
18:	learn: 500.2865936	total: 74.1m

# Stacked Rpart


In [ ]:

# Define base regressors
base_regressors = [
    ('decision_tree', DecisionTreeRegressor(max_depth=5, random_state=0)),
    ('linear_regression', LinearRegression()),
    ('svr', SVR(kernel='linear')),
    ('random_forest', RandomForestRegressor(n_estimators=50, random_state=0))
]

# Define the meta-regressor
meta_regressor = DecisionTreeRegressor()

# Create the stacked regressor
stacked_regressor = StackingRegressor(estimators=base_regressors, final_estimator=meta_regressor)

# Fit the model to the training data
stacked_regressor.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = stacked_regressor.predict(X_test_scaled)

# Calculate evaluation metrics
mae = mean_absoiute_error(y_test, y_pred)
rmse = r_mse(y_test, y_pred) # RMSE
r2 = r_squared(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Absolute Error (MAE): 0.35612432432432434
Root Mean Squared Error (RMSE): 1.6700274610139023
R-squared (R2) Score: 0.1611732471949724


#Stacking using decision tree


In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Define base models
dt1 = DecisionTreeRegressor(max_depth=3, random_state=42)
dt2 = DecisionTreeRegressor(max_depth=5, random_state=42)

# Fit the base models on the training data
dt1.fit(X_train, y_train)
dt2.fit(X_train, y_train)

# Make predictions on the test data using the base models
dt1_y_pred = dt1.predict(X_test)
dt2_y_pred = dt2.predict(X_test)

# Stack the predictions from the base models into a new feature matrix
stacking_train = np.column_stack((dt1.predict(X_train), dt2.predict(X_train)))
stacking_test = np.column_stack((dt1_y_pred, dt2_y_pred))

# Fit a meta model on the new feature matrix and the training labels
meta_model = RandomForestRegressor(n_estimators=100, random_state=42)
meta_model.fit(stacking_train, y_train)

# Make predictions on the test data using the meta model
y_pred = meta_model.predict(stacking_test)

# Calculate evaluation metrics
mae = mean_absoiute_error(y_test, y_pred)
rmse = r_mse(y_test, y_pred) # RMSE
r2 = r_squared(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

Mean Absolute Error (MAE): 0.3529445123485307
Root Mean Squared Error (RMSE): 1.6590212601842305
R-squared (R2) Score: 0.17219327352791258


<ipython-input-17-da23d9c78fbd>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  meta_model.fit(stacking_train, y_train)


# **Stacking using SVM**

In [ ]:




# Define base models
svm1 = SVR(kernel='linear')
svm2 = SVR(kernel='rbf')

# Fit the base models on the training data
svm1.fit(X_train, y_train)
svm2.fit(X_train, y_train)

# Make predictions on the test data using the base models
svm1_y_pred = svm1.predict(X_test)
svm2_y_pred = svm2.predict(X_test)

# Stack the predictions from the base models into a new feature matrix
stacking_train = np.column_stack((svm1.predict(X_train), svm2.predict(X_train)))
stacking_test = np.column_stack((svm1_y_pred, svm2_y_pred))

# Fit a meta model on the new feature matrix and the training labels
meta_model = RandomForestRegressor(n_estimators=100, random_state=42)
meta_model.fit(stacking_train, y_train)

# Make predictions on the test data using the meta model
y_pred = meta_model.predict(stacking_test)

# Calculate evaluation metrics
mae = mean_absoiute_error(y_test, y_pred)
rmse = r_mse(y_test, y_pred) # RMSE
r2 = r_squared(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Absolute Error (MAE): 0.3586934324324324
Root Mean Squared Error (RMSE): 1.654258613225754
R-squared (R2) Score: 0.17693931534092255


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-18-6a347530a7ec>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  meta_model.fit(stacking_train, y_train)


# **Stacking using Random Forest**

In [ ]:




# Define base models
rf1 = RandomForestRegressor(n_estimators=10, random_state=42)
rf2 = RandomForestRegressor(n_estimators=50, random_state=42)

# Fit the base models on the training data
rf1.fit(X_train, y_train)
rf2.fit(X_train, y_train)

# Make predictions on the test data using the base models
rf1_y_pred = rf1.predict(X_test)
rf2_y_pred = rf2.predict(X_test)

# Stack the predictions from the base models into a new feature matrix
stacking_train = np.column_stack((rf1.predict(X_train), rf2.predict(X_train)))
stacking_test = np.column_stack((rf1_y_pred, rf2_y_pred))

# Fit a meta model on the new feature matrix and the training labels
meta_model = RandomForestRegressor(n_estimators=100, random_state=42)
meta_model.fit(stacking_train, y_train)

# Make predictions on the test data using the meta model
y_pred = meta_model.predict(stacking_test)

# Calculate evaluation metrics
mae = mean_absoiute_error(y_test, y_pred)
rmse = r_mse(y_test, y_pred) # RMSE
r2 = r_squared(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


<ipython-input-19-ac4cd344be02>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf1.fit(X_train, y_train)
<ipython-input-19-ac4cd344be02>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf2.fit(X_train, y_train)
<ipython-input-19-ac4cd344be02>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  meta_model.fit(stacking_train, y_train)


Mean Absolute Error (MAE): 0.353311108108108
Root Mean Squared Error (RMSE): 1.6531707937487314
R-squared (R2) Score: 0.17802142796080056


# Stacked XGBoost

In [ ]:
from xgboost import XGBRegressor


# Create the base models (XGBoost)
xgb_model1 = XGBRegressor(n_estimators=100, random_state=42)
xgb_model2 = XGBRegressor(n_estimators=100, random_state=42)

# Train the base models
xgb_model1.fit(X_train, y_train)
xgb_model2.fit(X_train, y_train)

# Make predictions with the base models
xgb_predictions1 = xgb_model1.predict(X_test)
xgb_predictions2 = xgb_model2.predict(X_test)

# Create a meta-model (XGBoost) to combine predictions
meta_model = XGBRegressor(n_estimators=100, random_state=42)

# Combine the predictions from base models as meta-features
meta_features = pd.DataFrame({'XGB1': xgb_predictions1, 'XGB2': xgb_predictions2})

# Train the meta-model on the predictions
meta_model.fit(meta_features, y_test)

# Make predictions with the meta-model
meta_predictions = meta_model.predict(meta_features)

# Calculate the MAE for the ensemble model
# mae = mean_absolute_error(y_test, meta_predictions)
# mse = mean_squared_error(y_test, meta_predictions)
# rmse = np.sqrt(mse)
r2 = r2_score(y_test, meta_predictions)

mae = mean_absoiute_error(y_test, y_pred)
rmse = r_mse(y_test, y_pred) # RMSE
# r2 = r_squared(y_test, y_pred)

print(f"Ensemble MAE: {mae}")

print(f"Ensemble RMSE: {rmse}")
print(f"Ensemble R-squared: {r2}")


Ensemble MAE: 0.010733592068965516
Ensemble RMSE: 0.019013679567842095
Ensemble R-squared: 0.9999999821830945


# Stacked GradientBoost

In [ ]:
# Define base models
base_model1 = GradientBoostingRegressor(n_estimators=100, random_state=42)
base_model2 = GradientBoostingRegressor(n_estimators=200, random_state=42)

# Define the stacked model
stacked_model = StackingRegressor(
    estimators=[
        ('model1', base_model1),
        ('model2', base_model2)
    ],
    final_estimator=GradientBoostingRegressor(n_estimators=50, random_state=42)
)

# Fit the stacked model
stacked_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = stacked_model.predict(X_test)


# Calculate evaluation metrics
mae = mean_absoiute_error(y_test, y_pred)
rmse = r_mse(y_test, y_pred) # RMSE
r2 = r_squared(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Absolute Error (MAE): 0.38986405066915536
Root Mean Squared Error (RMSE): 1.8000958379512937
R-squared (R2) Score: 0.025422673022146403


#Stacked nnet

In [ ]:
from sklearn.neural_network import MLPRegressor


# Define base regressors (neural networks)
base_regressors = [
    ('mlp_1', MLPRegressor(hidden_layer_sizes=(50,), max_iter=1000, random_state=0)),
    ('mlp_2', MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=0)),
    ('mlp_3', MLPRegressor(hidden_layer_sizes=(50, 25), max_iter=1000, random_state=0))
]

# Define the meta-regressor
meta_regressor = MLPRegressor(hidden_layer_sizes=(50,), max_iter=1000, random_state=0)

# Create the stacked regressor
stacked_regressor = StackingRegressor(estimators=base_regressors, final_estimator=meta_regressor)

# Fit the model to the training data
stacked_regressor.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = stacked_regressor.predict(X_test_scaled)

# Calculate evaluation metrics
mae = mean_absoiute_error(y_test, y_pred)
rmse = r_mse(y_test, y_pred) # RMSE
r2 = r_squared(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklear

Mean Absolute Error (MAE): 0.13784274218002748
Root Mean Squared Error (RMSE): 0.45529924036974445
R-squared (R2) Score: 0.937652520735521


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


#Stacked glmnet

In [ ]:
from sklearn.linear_model import ElasticNet


# Define base regressors (GLMNet - ElasticNet)
base_regressors = [
    ('elastic_net_1', ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=0)),
    ('elastic_net_2', ElasticNet(alpha=0.01, l1_ratio=0.7, random_state=0)),
    ('elastic_net_3', ElasticNet(alpha=0.001, l1_ratio=0.9, random_state=0))
]

# Define the meta-regressor
meta_regressor = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=0)

# Create the stacked regressor
stacked_regressor = StackingRegressor(estimators=base_regressors, final_estimator=meta_regressor)

# Fit the model to the training data
stacked_regressor.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = stacked_regressor.predict(X_test_scaled)

# Calculate evaluation metrics
mae = mean_absoiute_error(y_test, y_pred)
rmse = r_mse(y_test, y_pred) # RMSE
r2 = r_squared(y_test, y_pred)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)


Mean Absolute Error (MAE): 0.13873050972877093
Root Mean Squared Error (RMSE): 0.45573384576369425
R-squared (R2) Score: 0.9375334364899288


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.891e+07, tolerance: 6.094e+04
  model = cd_fast.enet_coordinate_descent(


#Averaging

In [ ]:


# Define base models
models = [
    RandomForestRegressor(random_state=42),
    LinearRegression()
]

# Train base models and make predictions
predictions = []
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions.append(y_pred)

# Average predictions
avg_predictions = sum(predictions) / len(predictions)

# Calculate evaluation metrics
mae = mean_absoiute_error(y_test, avg_predictions)
rmse = r_mse(y_test, avg_predictions) # RMSE
r2 = r_squared(y_test, avg_predictions)


print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2) Score:", r2)

<ipython-input-24-17d517355a08>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


ValueError: y_true and y_pred have different number of output (1!=37)